In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyomo.environ import *

In [2]:
# Dados Entrada

ger = ['Termico_1', 'Termico_2', 'Termico_3']
Ng = len(ger)

wnd = ['Eolico_1', 'Eolico_2']
Nw = len(wnd)

loads = ['Carga_1', 'Carga_2']
Nc = len(loads)

bus = ['Barra_1', 'Barra_2']
Nb = len(bus)

demanda_energia = [40,100] # por carga

C_ener = [10,30,35] # $/MWh por gerador térmico
C_up = [16,13,10]   # $/MWh por gerador térmico
C_dw = [15,12,9]    # $/MWh por gerador térmico
C_shed = [200,200]  # $/MWh por carga
C_wind = [0, 0]        # $/MWh por gerador eólico

pmax_ter = [50,110,100] # potencia máxima geradores térmicos

# Previsão de produção da turbina eólica, considerando média ponderada dos cenários
p_wnd_forecast = [15, 30]

#Incerteza da eólica: Desvios
delta_wnd = [10, 20]

#Budget of Uncertainty
budget_uncertainty = 1.4

# Localização dos geradores térmicos
ger_loc = pd.DataFrame(np.array([[1,0], [1,0], [0,1]]),
                       columns=bus, index=ger)

# Localização dos geradores eólicos
wnd_loc = pd.DataFrame(np.array([[1,0], [0,1]]),
                       columns=bus, index=wnd)

# Localização das cargas
load_loc = pd.DataFrame(np.array([[1,0], [0,1]]),
                        columns=bus, index=loads)

# Matriz de conexões entre barras
conex = pd.DataFrame(np.array([[0,1], [1,0]]),
                     columns=bus, index=bus)

# Matriz de reatâncias das linhas
x_line = pd.DataFrame(np.array([[0,0.13], [0.13,0]]),
                      columns=bus, index=bus)

# Matriz de limites térmicos das linhas
t_line = pd.DataFrame(np.array([[0,100], [100,0]]),
                      columns=bus, index=bus)

In [3]:
def gerar_vertices(delta_w_max, gamma):
    """
    Gera os vértices de um conjunto de incerteza poliedral considerando múltiplas variáveis com restrição de budget de incerteza.

    Parâmetros:
        delta_w_max (list of float): Lista com os desvios máximos permitidos para cada variável.
        gamma (float): Budget of uncertainty (Γ).

    Retorna:
        list of tuples: Lista de vértices representados como tuplas.
    """
    vertices = []
    n = len(delta_w_max)

    # Para cada variável, fixamos ela em +ΔW_max e -ΔW_max
    for i in range(n):
        max_i = delta_w_max[i]

        # Tentamos fixar a variável i em +ΔW_max
        for sinal in [1, -1]:
            desvios = [0] * n  # Inicia todos os desvios como 0
            desvios[i] = sinal * max_i  # Fixa a variável i no valor máximo positivo ou negativo

            # Calcula o budget restante após fixar a variável i
            budget_restante = gamma - abs(desvios[i]) / delta_w_max[i]

            # Se o budget restante for suficiente, distribuir desvios nas outras variáveis
            for j in range(n):
                if j != i:
                    max_j = delta_w_max[j]
                    # Atribui o máximo possível permitido pelo budget restante
                    desvios[j] = budget_restante * max_j if budget_restante >= 1 else budget_restante * max_j
                    vertices.append(tuple(desvios))
                    desvios[j] = -desvios[j]  # Inverte o sinal para o outro vértice
                    vertices.append(tuple(desvios))
                    desvios[j] = 0  # Reseta para a próxima iteração

    return vertices

# Gera e exibe os vértices
vertices = gerar_vertices(delta_wnd, budget_uncertainty)
nvertices = len(vertices)
vert = [f'Vértice {i+1}' for i in range(nvertices)]

In [4]:
# Criar Modelo Concreto
model = ConcreteModel('mercado_conjunto_incerteza_renovavel')

#Criar Sets e seus parâmetros:

#Geradores Térmicos:
model.ger = Set(initialize=ger)
model.c_ener = Param(model.ger, initialize={ger[i]: C_ener[i] for i in range(Ng)})
model.c_up = Param(model.ger, initialize={ger[i]: C_up[i] for i in range(Ng)})
model.c_dw = Param(model.ger, initialize={ger[i]: C_dw[i] for i in range(Ng)})
model.pmax_ter = Param(model.ger, initialize={ger[i]: pmax_ter[i] for i in range(Ng)})

#Geradores Eólicos:
model.wnd = Set(initialize=wnd)
model.c_wind = Param(model.wnd, initialize={wnd[i]: C_wind[i] for i in range(Nw)})
model.p_wnd_forecast = Param(model.wnd, initialize={wnd[i]:p_wnd_forecast[i] for i in range(Nw)})

#Cargas
model.loads = Set(initialize=loads)
model.dem = Param(model.loads, initialize={loads[i]: demanda_energia[i] for i in range(Nc)})
model.c_shed = Param(model.loads, initialize={loads[i]: C_shed[i] for i in range(Nc)})

#Vértices
model.vert  = Set(initialize=vert)
# Inicializa o dicionário para armazenar os valores ajustados de p_wnd_real
p_wnd_real_data = {}

# Calcula p_wnd_real para cada gerador eólico e cada vértice de incerteza
for v, desvios in enumerate(vertices):
    for w, wnd_gen in enumerate(wnd):
        # Ajusta a previsão com o desvio do vértice correspondente
        p_wnd_real_data[(wnd_gen, f'Vértice {v+1}')] = p_wnd_forecast[w] + desvios[w]

# Define o parâmetro p_wnd_real no modelo Pyomo com os dados calculados
model.p_wnd_real = Param(model.wnd, model.vert, initialize=p_wnd_real_data)

#Barras e localizações:
model.bus = Set(initialize=bus)
model.ger_loc = Param(model.ger, model.bus, initialize=lambda model, g, b: ger_loc.loc[g, b], default=0, mutable=False)
model.wnd_loc = Param(model.wnd, model.bus, initialize=lambda model, w, b: wnd_loc.loc[w, b], default=0, mutable=False)
model.load_loc = Param(model.loads, model.bus, initialize=lambda model, l, b: load_loc.loc[l, b], default=0, mutable=False)
model.conex = Param(model.bus, model.bus, initialize=lambda model, b1, b2: conex.loc[b1, b2], default=0, mutable=False)
model.x_line = Param(model.bus, model.bus, initialize=lambda model, b1, b2: x_line.loc[b1, b2], default=0, mutable=False)
model.t_line = Param(model.bus, model.bus, initialize=lambda model, b1, b2: t_line.loc[b1, b2], default=0, mutable=False)

In [5]:
# Declarar Variáveis
model.P_ger = Var(model.ger, domain=NonNegativeReals) # Produção dos Geradores Térmicos
model.P_wnd = Var(model.wnd, domain=NonNegativeReals) # Produção dos Geradores Eólicos
model.R_up = Var(model.ger, domain=NonNegativeReals) # Reserva de Subida da Capacidade dos Produtores
model.R_dw = Var(model.ger, domain=NonNegativeReals) # Reserva de Descida da Capacidade dos Produtores
model.P_spill = Var(model.wnd,model.vert, domain=NonNegativeReals) # Descarte de Energia da Eólica (Spillage)
model.Reg_up = Var(model.ger,model.vert, domain=NonNegativeReals) # Ativação Reserva Subida Geradores Térmicos
model.Reg_dw = Var(model.ger,model.vert, domain=NonNegativeReals) # Ativação Reserva Descida Geradores Térmicos
model.L_shed = Var(model.loads,model.vert, domain=NonNegativeReals) # Corte de Carga
model.Theta_DA = Var(model.bus,domain=Reals) # Ângulos das Barras
model.Theta_RT = Var(model.bus,model.vert, domain=Reals) # Ângulos das Barras
model.Beta = Var(domain=NonNegativeReals, initialize=0) # Beta

In [6]:
def upper_bounds(model, ger): # restrição limite superior (ger)
    return model.P_ger[ger] + model.R_up[ger]<= model.pmax_ter[ger]
model.UpperBound = Constraint(model.ger, rule=upper_bounds)

def lower_bounds(model, ger): # restrição limite inferior (ger)
    return model.P_ger[ger] - model.R_dw[ger]>= 0
model.LowerBound = Constraint(model.ger, rule=lower_bounds)

def ger_wnd(model, wnd): # restrição geração eólica (wnd)
    return model.P_wnd[wnd] == model.p_wnd_forecast[wnd]
model.GerWnd = Constraint(model.wnd, rule=ger_wnd)

def day_ahead_balance(model, bus):  #Restrição do balanço de energia no mercado diário
    thermal_generation = sum(model.P_ger[g]*model.ger_loc[g,bus] for g in model.ger)
    wind_generation = sum(model.P_wnd[w]*model.wnd_loc[w,bus] for w in model.wnd)
    load = sum(model.dem[l]*model.load_loc[l,bus] for l in model.loads)
    line_flow = sum((model.Theta_DA[bus] - model.Theta_DA[j])/ model.x_line[bus,j] for j in model.bus if model.conex[bus,j]==1)
    return thermal_generation + wind_generation - load == line_flow
model.DA_Mkt = Constraint(model.bus, rule=day_ahead_balance)

def day_ahead_flow_limits(model, i, j): # restrição limite térmico da linha - tipo 1 (bus,bus)
    if i != j and model.conex[i,j] == 1:
        return (model.Theta_DA[i] - model.Theta_DA[j])/model.x_line[i,j] <= model.t_line[i,j]
    else:
        return Constraint.Skip
model.DA_Flow_Limits = Constraint(model.bus, model.bus, rule=day_ahead_flow_limits)

def operation_flow_limits(model, i, j, vert): # restrição limite térmico da linha - tipo 2 (bus,bus,scen)
     if i != j and model.conex[i,j] == 1:
        return (model.Theta_RT[i,vert] - model.Theta_RT[j,vert])/model.x_line[i,j] <= model.t_line[i,j]
     else:
        return Constraint.Skip
model.Operation_Flow_Limits = Constraint(model.bus, model.bus, model.vert, rule=operation_flow_limits)

def non_ant_up(model, ger, scen): # restrição non-anticipativity subida gerador (ger,scen)
    return model.Reg_up[ger,scen] <= model.R_up[ger]
model.NonAntUp = Constraint(model.ger, model.vert, rule=non_ant_up)

def non_ant_dw(model, ger, scen): # restrição non-anticipativity descida gerador (ger,scen)
    return model.Reg_dw[ger,scen] <= model.R_dw[ger]
model.NonAntDw = Constraint(model.ger, model.vert, rule=non_ant_dw)

def non_ant_shed(model, loads, scen): # restrição non-anticipativity corte carga (load,scen)
    return model.L_shed[loads,scen] <= model.dem[loads]
model.NonAntShed = Constraint(model.loads, model.vert, rule=non_ant_shed)

def non_ant_spillage(model, wnd, vert):  # restrição non-anticipativity spillage (wnd,scen)
    return model.P_spill[wnd,vert]<=model.p_wnd_real[wnd,vert]
model.NonAntSp = Constraint(model.wnd, model.vert, rule=non_ant_spillage)

def operation_balance(model, bus, scen): # restrição balanço de energia (bus,scen)
    thermal_reserve_ativation = sum((model.Reg_up[g,scen]-model.Reg_dw[g,scen])*model.ger_loc[g,bus] for g in model.ger)
    wind = sum((model.p_wnd_real[w,scen] - model.P_wnd[w] - model.P_spill[w,scen])*model.wnd_loc[w, bus] for w in model.wnd)
    load = sum(model.L_shed[l,scen]*model.load_loc[l,bus] for l in model.loads)
    line_flow = sum((model.Theta_RT[bus,scen] - model.Theta_DA[bus] + model.Theta_DA[j] - model.Theta_RT[j, scen])/model.x_line[bus,j] for j in model.bus if model.conex[bus, j] == 1)
    return thermal_reserve_ativation + wind + load == line_flow
model.Operation_Balance = Constraint(model.bus, model.vert, rule=operation_balance)

def beta_restr(model, vertices): # restrição beta
    custo_reserva_ativada = sum((model.Reg_up[g, vertices] - model.Reg_dw[g, vertices]) * model.c_ener[g] for g in model.ger)
    custo_wnd_real = sum((model.p_wnd_real[w,vertices] - model.P_wnd[w] - model.P_spill[w, vertices])*model.c_wind[w] for w in model.wnd)
    custo_carga_cortada = sum(model.L_shed[l, vertices] * model.c_shed[l] for l in model.loads)
    custo_beta = custo_reserva_ativada + custo_wnd_real + custo_carga_cortada
    return custo_beta <= model.Beta
model.Beta_restr = Constraint(model.vert, rule=beta_restr)


In [7]:
##Função Objetivo:
def fob(model):
    #Parte Determinística:
    custo_energia = sum(model.c_ener[g]*model.P_ger[g] for g in model.ger)
    custo_reserva_subida = sum(model.c_up[g]*model.R_up[g] for g in model.ger)
    custo_reserva_descida = sum(model.c_dw[g]*model.R_dw[g] for g in model.ger)
    custo_wnd = sum(model.c_wind[w]*model.P_wnd[w] for w in model.wnd)
    custo_deterministico = custo_energia + custo_reserva_subida + custo_reserva_descida + custo_wnd

    #Beta
    custo_beta = model.Beta

    custo_total = custo_deterministico + custo_beta
    return custo_total

model.obj = Objective(rule=fob, sense=minimize)

In [8]:
solver = SolverFactory('glpk')
# solver = SolverFactory('ipopt',executable='/content/ipopt')

resultados = solver.solve(model, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\gabri\AppData\Local\Temp\tmpbcexxvin.glpk.raw --wglp C:\Users\gabri\AppData\Local\Temp\tmpkompcquu.glpk.glp
 --cpxlp C:\Users\gabri\AppData\Local\Temp\tmpxfd2zf6t.pyomo.lp
Reading problem data from 'C:\Users\gabri\AppData\Local\Temp\tmpxfd2zf6t.pyomo.lp'...
132 rows, 110 columns, 419 non-zeros
944 lines were read
Writing problem data to 'C:\Users\gabri\AppData\Local\Temp\tmpkompcquu.glpk.glp'...
824 lines were written
GLPK Simplex Optimizer, v4.65
132 rows, 110 columns, 419 non-zeros
Preprocessing...
98 rows, 92 columns, 351 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  2.000e+02  ratio =  2.000e+02
GM: min|aij| =  4.729e-01  max|aij| =  2.115e+00  ratio =  4.472e+00
EQ: min|aij| =  2.236e-01  max|aij| =  1.000e+00  ratio =  4.472e+00
Constructing initial basis...
Size of triangular part is 98
      0: obj =   3.200000000e+03 inf =   8.556e+01 (8)
     13: obj =   4.188000000e+

In [9]:
# Relatório dos resultados de otimização

print('Status Final do Problema de Otimização:', resultados.solver.status, '\n')
print('Condição de Término:', resultados.solver.termination_condition, '\n')
print('Resultado Função Objetivo:', value(model.obj), '\n')
print('Variáveis de Decisão: \n')

Status Final do Problema de Otimização: ok 

Condição de Término: optimal 

Resultado Função Objetivo: 2882.0 

Variáveis de Decisão: 



In [11]:
# prompt: printe os valores das variáveis desse problema de otimização

print("Produção dos Geradores Térmicos:")
for g in model.ger:
  print(f"  Gerador {g}: {model.P_ger[g].value} MWh")

print("\nProdução dos Geradores Eólicos:")
for w in model.wnd:
  print(f"  Gerador {w}: {model.P_wnd[w].value} MWh")

print("\nReserva de Subida da Capacidade dos Produtores:")
for g in model.ger:
  print(f"  Gerador {g}: {model.R_up[g].value} MW")

print("\nReserva de Descida da Capacidade dos Produtores:")
for g in model.ger:
  print(f"  Gerador {g}: {model.R_dw[g].value} MW")

print("\nDescarte de Energia da Eólica (Spillage):")
for w in model.wnd:
  for s in model.vert:
    print(f"  Gerador {w}, {s}: {model.P_spill[w,s].value} MW")

print("\nAtivação Reserva Subida Geradores Térmicos:")
for g in model.ger:
  for s in model.vert:
    print(f"  Gerador {g}, {s}: {model.Reg_up[g,s].value} MW")

print("\nAtivação Reserva Descida Geradores Térmicos:")
for g in model.ger:
  for s in model.vert:
    print(f"  Gerador {g}, {s}: {model.Reg_dw[g,s].value} MW")

print("\nCorte de Carga:")
for l in model.loads:
  for s in model.vert:
    print(f"  {l}, {s}: {model.L_shed[l,s].value} MWh")

print("\nÂngulos das Barras (Mercado Diário):")
for b in model.bus:
  print(f"  {b}: {model.Theta_DA[b].value} rad")

print("\nÂngulos das Barras (Tempo Real):")
for b in model.bus:
  for s in model.vert:
    print(f"  {b}, {s}: {model.Theta_RT[b,s].value} rad")

print("\nBeta:")
print(f"  R$ {model.Beta.value}")

Produção dos Geradores Térmicos:
  Gerador Termico_1: 50.0 MWh
  Gerador Termico_2: 45.0 MWh
  Gerador Termico_3: -1.09590264382537e-14 MWh

Produção dos Geradores Eólicos:
  Gerador Eolico_1: 15.0 MWh
  Gerador Eolico_2: 30.0 MWh

Reserva de Subida da Capacidade dos Produtores:
  Gerador Termico_1: 3.45042665029279e-14 MW
  Gerador Termico_2: 24.0 MW
  Gerador Termico_3: 2.46459046449485e-15 MW

Reserva de Descida da Capacidade dos Produtores:
  Gerador Termico_1: 0.0 MW
  Gerador Termico_2: 0.0 MW
  Gerador Termico_3: 0.0 MW

Descarte de Energia da Eólica (Spillage):
  Gerador Eolico_1, Vértice 1: 18.0 MW
  Gerador Eolico_1, Vértice 2: 1.99999999999999 MW
  Gerador Eolico_1, Vértice 3: 0.0 MW
  Gerador Eolico_1, Vértice 4: 0.0 MW
  Gerador Eolico_1, Vértice 5: 19.0 MW
  Gerador Eolico_1, Vértice 6: 11.0 MW
  Gerador Eolico_1, Vértice 7: 0.0 MW
  Gerador Eolico_1, Vértice 8: 0.0 MW
  Gerador Eolico_2, Vértice 1: 0.0 MW
  Gerador Eolico_2, Vértice 2: 0.0 MW
  Gerador Eolico_2, Vértice 